# "Fallecidos confirmados por comuna"
> Fallecidos confirmados por comuna.

- toc: true 
- badges: true
- comments: true
- author: Alonso Silva Allende
- categories: [jupyter]
- image: images/diagram.png

In [1]:
#hide
import numpy as np
import pandas as pd
import altair as alt
from IPython.display import display, Markdown, display_html, HTML

In [2]:
#hide
update_date = pd.to_datetime('today') - pd.offsets.Hour(19)

In [3]:
#hide_input
display(Markdown(f"Última actualización: {update_date.strftime('%d/%m/%Y')}."))

Última actualización: 28/06/2020.

# Evolución del total de fallecidos confirmados por comuna

In [4]:
#collapse-hide
import numpy as np
import pandas as pd
import altair as alt
from IPython.display import display, Markdown, display_html, HTML

#download the data
county_deaths_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto38/CasosFallecidosPorComuna.csv")
#select Metropolitan region
county_deaths_RM_raw = county_deaths_raw.query(
    "Region=='Metropolitana'").drop(
    columns=["Region", "Codigo region", "Codigo comuna"]).set_index("Comuna")
#first and last report date
first_date = county_deaths_RM_raw.T.iloc[1].name
last_date = county_deaths_raw.T.iloc[-1].name
#build dataframe with all the dates since first date to last date
county_deaths_RM = pd.DataFrame()
county_deaths_RM["Fecha"] = pd.date_range(start=first_date, end=last_date).strftime('%Y-%m-%d')
county_deaths_RM = county_deaths_RM.set_index("Fecha")
#transpose the dataframe
county_deaths_RM_raw = county_deaths_RM_raw.T
#build the dataframe with all the data from the reports
county_deaths_RM[list(county_deaths_RM_raw.columns)] = county_deaths_RM_raw[list(county_deaths_RM_raw.columns)]

#data in long form to plot it on Altair
county_deaths_RM_long_form = county_deaths_RM.drop(
    columns="Total").reset_index().melt("Fecha", var_name="Comuna", value_name="Fallecidos confirmados")

#plot the data
input_dropdown = alt.binding_select(options=county_deaths_RM_long_form['Comuna'].unique())
selection1 = alt.selection_single(fields=['Comuna'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['Comuna'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('Comuna:N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

bars1 = alt.Chart(county_deaths_RM_long_form.fillna(0)).mark_bar().encode(
    x=alt.X("Fecha", axis=alt.Axis(title="", domain=False, tickSize=0)),
    y=alt.Y("Fallecidos confirmados", axis=alt.Axis(title="número de fallecidos confirmados")),
    tooltip = ["Fecha", "Comuna", "Fallecidos confirmados"],
    color = color
).properties(
    title="Total fallecidos confirmados por comuna de la R.M.",
    width=700).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

bars1

alt.Chart(...)

# Evolución del total de fallecidos confirmados por comuna (interpolación)

In [5]:
#collapse-hide
#linear interpolation
county_deaths_RM = np.round(county_deaths_RM.interpolate(method="linear"))

#data in long form to plot it on altair
county_deaths_RM_long_form = county_deaths_RM.drop(columns="Total").reset_index().melt("Fecha", var_name="Comuna", value_name="Fallecidos confirmados")
#plot the data
input_dropdown = alt.binding_select(options=county_deaths_RM_long_form['Comuna'].unique())
selection1 = alt.selection_single(fields=['Comuna'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['Comuna'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('Comuna:N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

bars1 = alt.Chart(county_deaths_RM_long_form.fillna(0)).mark_bar().encode(
    x=alt.X("Fecha", axis=alt.Axis(title="", domain=False, tickSize=0)),
    y=alt.Y("Fallecidos confirmados", axis=alt.Axis(title="número de fallecidos confirmados")),
    tooltip = ["Fecha", "Comuna", "Fallecidos confirmados"],
    color = color
).properties(
    title="Total fallecidos confirmados por comuna de la R.M.",
    width=700).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

bars1

alt.Chart(...)

# Evolución de nuevos fallecidos confirmados por comuna (interpolación)

In [6]:
#collapse-hide
new_county_deaths_RM = county_deaths_RM.diff().drop("2020-06-12")

#data in long form to plot it on altair
new_county_deaths_RM_long_form = new_county_deaths_RM.drop(columns="Total").reset_index().melt("Fecha", var_name="Comuna", value_name="Fallecidos confirmados")
#plot the data
input_dropdown = alt.binding_select(options=new_county_deaths_RM_long_form['Comuna'].unique())
selection1 = alt.selection_single(fields=['Comuna'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['Comuna'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('Comuna:N', scale=alt.Scale(scheme='tableau20'), legend=None),
                    alt.value('lightgray'))

bars1 = alt.Chart(new_county_deaths_RM_long_form.fillna(0)).mark_bar().encode(
    x=alt.X("Fecha", axis=alt.Axis(title="", domain=False, tickSize=0)),
    y=alt.Y("Fallecidos confirmados", axis=alt.Axis(title="número de fallecidos confirmados")),
    tooltip = ["Fecha", "Comuna", "Fallecidos confirmados"],
    color = color
).properties(
    title="Nuevos fallecidos confirmados por comuna de la R.M.",
    width=700).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

bars1

alt.Chart(...)

In [7]:
#hide
new_county_deaths_RM.sum(axis=1)

Fecha
2020-06-13    277.0
2020-06-14    268.0
2020-06-15    277.0
2020-06-16    335.0
2020-06-17    342.0
2020-06-18    336.0
2020-06-19    335.0
2020-06-20    174.0
2020-06-21    182.0
2020-06-22    174.0
2020-06-23    174.0
2020-06-24    508.0
2020-06-25    514.0
2020-06-26    516.0
2020-06-27    514.0
2020-06-28    508.0
dtype: float64

In [8]:
#hide
print(pd.to_datetime(last_date)-pd.DateOffset(13))
print(pd.to_datetime(last_date)-pd.DateOffset(7))
print(pd.to_datetime(last_date)-pd.DateOffset(6))
print(pd.to_datetime(last_date))

2020-06-15 00:00:00
2020-06-21 00:00:00
2020-06-22 00:00:00
2020-06-28 00:00:00


In [9]:
#hide
deaths_last_week = pd.Series(dtype='float64')
deaths_two_weeks_ago = pd.Series(dtype='float64')
for comuna in new_county_deaths_RM.columns:
    deaths_last_week[comuna] = \
    new_county_deaths_RM.loc[(pd.to_datetime(last_date)-pd.DateOffset(6)).strftime('%Y-%m-%d'):last_date, comuna].sum()
    deaths_two_weeks_ago[comuna] = \
    new_county_deaths_RM.loc[(pd.to_datetime(last_date)-pd.DateOffset(13)).strftime('%Y-%m-%d'):(pd.to_datetime(last_date)-pd.DateOffset(7)).strftime('%Y-%m-%d'), comuna].sum()    #print(f"{comuna}: {new_county_deaths_RM.loc[(pd.to_datetime(last_date)-pd.DateOffset(6)).strftime('%Y-%m-%d'):last_date, comuna].sum()}")

In [20]:
#hide
deaths = pd.DataFrame()
deaths["Comuna"] = new_county_deaths_RM.columns
deaths = deaths.set_index("Comuna")

In [23]:
#hide
deaths["Hace dos semanas"] = deaths_two_weeks_ago
deaths["Última semana"] = deaths_last_week
deaths["Diferencia"] = deaths_last_week - deaths_two_weeks_ago
deaths["Incremento"] = deaths_last_week/deaths_two_weeks_ago-1
deaths["Texto"] = [f"{np.round(100*((deaths_last_week/deaths_two_weeks_ago).iloc[i]-1), decimals=1)}%" for i in np.arange(len(deaths))]

In [24]:
#hide
deaths.head()

,Hace dos semanas,Última semana,Diferencia,Incremento,Texto
Comuna,,,,,
Alhue,1.0,0.0,-1.0,-1.000000,-100.0%
Buin,8.0,13.0,5.0,0.625000,62.5%
Calera de Tango,2.0,1.0,-1.0,-0.500000,-50.0%
Cerrillos,11.0,9.0,-2.0,-0.181818,-18.2%
Cerro Navia,26.0,58.0,32.0,1.230769,123.1%


# Incremento de fallecidos confirmados por semana

In [25]:
#hide_input
bars = alt.Chart(deaths.drop("Total").reset_index()).mark_bar(opacity=0.8, size=25).encode(
    x = alt.X('Diferencia', axis=alt.Axis(title='Diferencia')),
    y = alt.Y('Comuna', sort='-x'),
    tooltip = ['Comuna','Hace dos semanas', 'Última semana', 'Diferencia'],
    color = alt.Color('Comuna', scale=alt.Scale(scheme='tableau20'), legend=None),
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to the right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Diferencia')
)

bars.properties(
    title = f"Incremento de fallecidos confirmados por semana por comuna de la R.M. al {pd.to_datetime(last_date).strftime('%d/%m')}",
    width = 600,
    height = alt.Step(30)
) + text

alt.LayerChart(...)

# Porcentaje de incremento de fallecidos confirmados por semana

In [27]:
#hide_input
bars = alt.Chart(deaths.drop("Total").reset_index()).mark_bar(opacity=0.8, size=25).encode(
    x = alt.X('Incremento', axis=alt.Axis(format='%', title='Incremento')),
    y = alt.Y('Comuna', sort='-x'),
    tooltip = ['Comuna','Hace dos semanas', 'Última semana', 'Incremento'],
    color = alt.Color('Comuna', scale=alt.Scale(scheme='tableau20'), legend=None),
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to the right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Texto')
)

bars.properties(
    title = f"Porcentaje de incremento de fallecidos confirmados por semana por comuna de la R.M. al {pd.to_datetime(last_date).strftime('%d/%m')}",
    width = 600,
    height = alt.Step(30)
) + text

alt.LayerChart(...)